# map

In [3]:
import os

In [1]:
import findspark
findspark.init()
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
nRdd = spark.sparkContext.parallelize([1, 2, 3, 4])
squared = nRdd.map(lambda x: x * x)
print (squared)

PythonRDD[1] at RDD at PythonRDD.scala:53


In [5]:
print(squared.collect())

[1, 4, 9, 16]


In [6]:
myRdd4 = spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_2cols.csv"))

In [64]:
myRdd4.take(5)

['35, 2', '40, 27', '12, 38', '15, 31', '21, 1']

In [8]:
myRdd5 = myRdd4.map(lambda line: line.split(','))
myRdd5.take(5)

[['35', ' 2'], ['40', ' 27'], ['12', ' 38'], ['15', ' 31'], ['21', ' 1']]

In [9]:
x=['35', '2']
y=list()
for i in x:
    y.append(int(i))
print(y)

[35, 2]


In [10]:
x=['35', '2']
y=list()
for i in x:
    y.append(int(i))
print(y)

[35, 2]

[35, 2]

# 문자열 숫자를 정수형으로 변환

In [13]:
x=['35', '2']
[int(i) for i in x]

[35, 2]

In [14]:
myRdd6 = myRdd5.map(lambda x: [int(i) for i in x])
myRdd6.take(5)

[[35, 2], [40, 27], [12, 38], [15, 31], [21, 1]]

# 단어 분리

In [15]:
myRdd2=spark.sparkContext\
    .textFile(os.path.join("data","ds_spark_wiki.txt"))

In [17]:
sentences=myRdd2.map(lambda x:x.split())

In [25]:
sentences.count()
# 단순 문장의 개수 출력

10

In [20]:
def mySplit(x):
    return x.split()
sentences2=myRdd2.map(mySplit)
sentences2.count()

10

In [24]:
sentences.take(3)

[['Wikipedia'],
 ['Apache',
  'Spark',
  'is',
  'an',
  'open',
  'source',
  'cluster',
  'computing',
  'framework.'],
 ['아파치', '스파크는', '오픈', '소스', '클러스터', '컴퓨팅', '프레임워크이다.']]

In [26]:
for line in sentences.collect():
    for word in line:
        print (word, end=" ")
    print ("\n-----")

Wikipedia 
-----
Apache Spark is an open source cluster computing framework. 
-----
아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. 
-----
Apache Spark Apache Spark Apache Spark Apache Spark 
-----
아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 
-----
Originally developed at the University of California, Berkeley's AMPLab, 
-----
the Spark codebase was later donated to the Apache Software Foundation, 
-----
which has maintained it since. 
-----
Spark provides an interface for programming entire clusters with 
-----
implicit data parallelism and fault-tolerance. 
-----


In [27]:
len("Apache Spark is an open source cluster computing framework")

58

In [28]:
myRdd2.map(lambda s:len(s)).collect()

[9, 59, 32, 51, 31, 72, 71, 30, 64, 46]

# 교체

In [29]:
myList=["this is","a line"]
_rdd=spark.sparkContext.parallelize(myList)

In [30]:
wordsRdd=_rdd.map(lambda x:x.split())
print (wordsRdd.collect())

[['this', 'is'], ['a', 'line']]


In [31]:
repRdd=_rdd.map(lambda x:x.replace("a","AA"))
repRdd.take(10)

['this is', 'AA line']

# 대소문자 변환

In [32]:
's'.upper()

'S'

In [33]:
upperRDD =wordsRdd.map(lambda x: x[0].upper())
print (upperRDD.collect())

['THIS', 'A']


In [34]:
upper2RDD =wordsRdd.map(lambda x: [i.upper() for i in x])
print (upper2RDD.collect())

[['THIS', 'IS'], ['A', 'LINE']]


# reduce

In [35]:
myRdd100 = spark.sparkContext.parallelize(range(1,101))
myRdd100.reduce(lambda subtotal, x: subtotal + x)

5050

# 단순통계기능

In [36]:
print ("sum: ", myRdd100.sum())
print ("min: ", myRdd100.min())
print ("max: ", myRdd100.max())
print ("standard deviation:", myRdd100.stdev())
print ("variance: ", myRdd100.variance())

sum:  5050
min:  1
max:  100
standard deviation: 28.86607004772212
variance:  833.25


# filter

In [37]:
myRdd_spark=myRdd2.filter(lambda line: "Spark" in line)
print ("How many lines having 'Spark': ",myRdd_spark.count())

How many lines having 'Spark':  4


In [42]:
myRdd_unicode = myRdd2.filter(lambda line: u"스파크" in line)
print (myRdd_unicode.take(2))

['아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.', '아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크']


In [40]:
stopwords = ['is','am','are','the','for','a', 'an', 'at']
myRdd_stop = myRdd2.flatMap(lambda x:x.split())\
                    .filter(lambda x: x not in stopwords)

In [41]:
for words in myRdd_stop.collect():
    print (words, end=' ')

Wikipedia Apache Spark open source cluster computing framework. 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다. Apache Spark Apache Spark Apache Spark Apache Spark 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크 Originally developed University of California, Berkeley's AMPLab, Spark codebase was later donated to Apache Software Foundation, which has maintained it since. Spark provides interface programming entire clusters with implicit data parallelism and fault-tolerance. 

# foreach()

In [43]:
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(lambda x: x + 1)

In [44]:
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).map(lambda x: x + 1).collect()

[2, 3, 4, 5, 6]

In [46]:
def f(x): print(x)
spark.sparkContext.parallelize([1, 2, 3, 4, 5]).foreach(f)

# pipeline
#### 함수를 연달아 적용

In [48]:
upper2list=wordsRdd.map(lambda x: [i.upper() for i in x]).collect()
print (type(upper2list))

<class 'list'>


In [49]:
wordsLength = wordsRdd\
    .map(len)\
    .collect()
print (wordsLength)

[2, 2]


# 파일에 쓰기

In [68]:
import findspark
findspark.init()
import pyspark
import os
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [72]:
!where hadoop.dll

정보: 제공된 패턴에 해당되는 파일을 찾지 못했습니다.


In [70]:
spark.sparkContext.parallelize(upper2list).saveAsTextFile("data/ds_spark_wiki_out3")

Py4JJavaError: An error occurred while calling o609.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/C:/Users/user/Desktop/3-2/Bigdata/201710773/data/ds_spark_wiki_out3 already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:298)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:414)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:108

In [59]:
!dir data

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 8084-31FA

 C:\Users\user\Desktop\3-2\Bigdata\201710773\data 디렉터리

2021-09-23  오후 03:09    <DIR>          .
2021-09-23  오후 03:09    <DIR>          ..
2021-09-10  오후 03:58            20,480 collection-0-5622828535532889306.wt
2021-09-10  오후 03:59            36,864 collection-2-5622828535532889306.wt
2021-09-10  오후 03:50             4,096 collection-4-5622828535532889306.wt
2021-09-10  오후 11:38    <DIR>          diagnostic.data
2021-09-17  오후 03:04               147 ds_spark_2cols.csv
2021-09-17  오후 12:31               583 ds_spark_wiki.txt
2021-09-23  오후 03:06    <DIR>          ds_spark_wiki_out
2021-09-23  오후 03:07    <DIR>          ds_spark_wiki_out2
2021-09-23  오후 03:09    <DIR>          ds_spark_wiki_out3
2021-09-10  오후 03:58            20,480 index-1-5622828535532889306.wt
2021-09-10  오후 03:59            36,864 index-3-5622828535532889306.wt
2021-09-10  오후 03:50             4,096 index-5-5622828535532889306.wt
2021-09-10  오후 03:59             4,09

In [60]:
_rdd=spark.sparkContext.textFile("data/ds_spark_wiki_out")
_rdd.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Ljava/lang/String;I)Z
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:645)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1230)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1435)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:493)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1868)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1910)
	at org.apache.hadoop.fs.FileSystem$4.<init>(FileSystem.java:2072)
	at org.apache.hadoop.fs.FileSystem.listLocatedStatus(FileSystem.java:2071)
	at org.apache.hadoop.fs.ChecksumFileSystem.listLocatedStatus(ChecksumFileSystem.java:700)
	at org.apache.hadoop.mapred.FileInputFormat.s

# groupBy
#### RDD를 어떤 기준에 따라 그룹으로 묶음

In [73]:
myRdd2.take(10)

['Wikipedia',
 'Apache Spark is an open source cluster computing framework.',
 '아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.',
 'Apache Spark Apache Spark Apache Spark Apache Spark',
 '아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크',
 "Originally developed at the University of California, Berkeley's AMPLab,",
 'the Spark codebase was later donated to the Apache Software Foundation,',
 'which has maintained it since.',
 'Spark provides an interface for programming entire clusters with',
 'implicit data parallelism and fault-tolerance.']

In [74]:
#myRdd_group=myRdd2.flatMap(lambda x:x.split()).groupBy(lambda x:w[0:2])
myRdd_group=myRdd2.groupBy(lambda x:x[0:2])

for (k,v) in myRdd_group.collect():
    print ("{}: {}".format(k, v))

Wi: <pyspark.resultiterable.ResultIterable object at 0x000001DA7BB8A9A0>
Ap: <pyspark.resultiterable.ResultIterable object at 0x000001DA7CBDEDF0>
아파: <pyspark.resultiterable.ResultIterable object at 0x000001DA7CBDEE80>
Or: <pyspark.resultiterable.ResultIterable object at 0x000001DA7CBDEA30>
th: <pyspark.resultiterable.ResultIterable object at 0x000001DA7CBDE730>
wh: <pyspark.resultiterable.ResultIterable object at 0x000001DA7CBDE820>
Sp: <pyspark.resultiterable.ResultIterable object at 0x000001DA7CBDEC40>
im: <pyspark.resultiterable.ResultIterable object at 0x000001DA7CBDEBB0>


In [76]:
#myRdd_group=myRdd2.flatMap(lambda x:x.split()).groupBy(lambda x:w[0:2])
myRdd_group=myRdd2.groupBy(lambda x:x[0:2])

for (k,v) in myRdd_group.collect():
    for eachValue in v:
        print ("{}: {}".format(k, eachValue))
    print ("-----")

Wi: Wikipedia
-----
Ap: Apache Spark is an open source cluster computing framework.
Ap: Apache Spark Apache Spark Apache Spark Apache Spark
-----
아파: 아파치 스파크는 오픈 소스 클러스터 컴퓨팅 프레임워크이다.
아파: 아파치 스파크 아파치 스파크 아파치 스파크 아파치 스파크
-----
Or: Originally developed at the University of California, Berkeley's AMPLab,
-----
th: the Spark codebase was later donated to the Apache Software Foundation,
-----
wh: which has maintained it since.
-----
Sp: Spark provides an interface for programming entire clusters with
-----
im: implicit data parallelism and fault-tolerance.
-----


In [78]:
_testList=[("Seoul",1),("Seoul",1),("Seoul",1),("Busan",1),("Busan",1),
           ("Seoul",1),("Busan",1),
           ("Seoul",1),("Seoul",1),("Busan",1),("Busan",1)]

In [79]:
_testRdd=spark.sparkContext.parallelize(_testList)

In [80]:
_testRdd.groupBy(lambda x:x[0]).collect()

[('Seoul', <pyspark.resultiterable.ResultIterable at 0x1da7bb8a6d0>),
 ('Busan', <pyspark.resultiterable.ResultIterable at 0x1da7bb915e0>)]

# mapValues
##### ResultIiterable을 리스트로 변환하여 볼 수 있도록 한다.

In [82]:
_testRdd.groupBy(lambda x:x[0]).mapValues(lambda x: list(x)).collect()

[('Seoul',
  [('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1)]),
 ('Busan',
  [('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1)])]

In [85]:
_testRdd.groupBy(lambda x:x[0]).mapValues(list).collect()

[('Seoul',
  [('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1),
   ('Seoul', 1)]),
 ('Busan',
  [('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1), ('Busan', 1)])]